In [2]:
from time import time
from math import log
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
df = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/train.csv')
df_test = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/test.csv')

In [4]:
doc = ''
doc1 = ''
doc0 = ''
for i in range(df.shape[0]):
    doc = doc + df.loc[i,'user_review'] + str(' ')
    if(df.loc[i,'user_suggestion']==0):
        doc0 = doc0 + df.loc[i,'user_review'] + str(' ')
    else:
        doc1 = doc1 + df.loc[i,'user_review'] + str(' ')

In [5]:
stop_words = stopwords.words('english')

tokens = word_tokenize(doc)
all_words = [token.lower() for token in tokens if token.isalpha()!=0]
all_words = [word for word in all_words if word not in stop_words]

tokens0 = word_tokenize(doc0)
all_words0 = [token.lower() for token in tokens0 if token.isalpha()!=0]
all_words0 = [word for word in all_words0 if word not in stop_words]

tokens1 = word_tokenize(doc1)
all_words1 = [token.lower() for token in tokens1 if token.isalpha()!=0]
all_words1 = [word for word in all_words1 if word not in stop_words]

In [6]:
freq_all_words = FreqDist(all_words)
freq_all_words0 = FreqDist(all_words0)
freq_all_words1 = FreqDist(all_words1)

word_features = []
word_features0 = []
word_features1 = []

word_features = word_features0 + word_features1

for i in range(5000):
    word_features.append(freq_all_words.most_common(5000)[i][0])

print(len(word_features))

47351
5000


In [6]:
def cond_prob(word,class_doc,main_doc_word_no,class_doc_word_no, alpha=1):
    n = len(class_doc.lower().split(word)) - 1
    prob = (n + alpha)/(class_doc_word_no + alpha*class_doc_word_no)
    return prob    

words = ['good', 'bad', 'cool', 'gross', 'paid', 'sucks','worst','best','yes','amazing','horrible']
for word in words:
    cond_p0 = cond_prob(word,doc0,len(all_words),len(all_words0))
    cond_p1 = cond_prob(word,doc1,len(all_words),len(all_words1))
    if(cond_p1 > cond_p0):
        print(word,"Good_review")
    else:
        print(word,"Bad_review")

good Good_review
bad Bad_review
cool Good_review
gross Bad_review
paid Bad_review
sucks Bad_review
worst Bad_review
best Good_review
yes Good_review
amazing Good_review
horrible Bad_review


In [7]:
def NB_prob(paragraph,doc1,doc0,p0,p1,main_doc_word_no,class1_doc_word_no,class0_doc_word_no,alpha=1):
    t0 = time()
    log_cond_p1_para = log(p1)
    log_cond_p0_para = log(p0)
    words = set([word for word in word_tokenize(paragraph.lower())])
    
    for w in words:
        log_cond_p1_para = log_cond_p1_para + log(cond_prob(w, doc1, main_doc_word_no, class1_doc_word_no,alpha))
        log_cond_p0_para = log_cond_p0_para + log(cond_prob(w, doc0, main_doc_word_no, class0_doc_word_no,alpha))
    
    print(paragraph[:5])
    if(log_cond_p1_para>=log_cond_p0_para):
        print("Recommend")
        return 1
    else:
        print("Nope")
        return 0

In [8]:
p1 = df['user_suggestion'].mean()
p0 = 1 - p1

NB_prob('Bad bad one, i don\'t know what to say about this one. it is absolutely terrible',doc1,doc0,p0,p1,len(all_words),len(all_words1),len(all_words0))
NB_prob('his is the best game I have ever played. I\'m really excited for the next' ,doc1,doc0,p0,p1,len(all_words),len(all_words1),len(all_words0))

Bad b
Nope
his i
Recommend


1

In [9]:
data = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/nlp_sub2.csv')
df_test['user_suggestion'] = data['user_suggestion']
# df_test['user_suggestion'] = pd.Series(np.zeros(df_test.shape[0]))

In [10]:
df_test['user_suggestion'][7800:] = df_test['user_review'][7800:].apply(lambda x: int(NB_prob(x,doc1,doc0,p0,p1,len(all_words),len(all_words1),len(all_words0))))

Early
Recommend
After
Nope
I was
Recommend
Over 
Nope
Early
Nope
Early
Nope
Early
Recommend
Early
Nope
Early
Nope
Early
Nope
Early
Nope
this 
Recommend
Early
Nope
Early
Recommend
Early
Nope
Early
Nope
Early
Nope
Good 
Recommend
Early
Nope
Best 
Nope
Early
Nope
Early
Nope
i pai
Nope
I bou
Nope
Early
Recommend
F-U  
Nope
Let's
Nope
Early
Nope
Early
Nope
I bou
Nope
I JUS
Nope
Early
Recommend
uhm, 
Nope
Early
Nope
H1Z1 
Nope
Reall
Nope
goofy
Recommend
If I 
Nope
The g
Nope
Early
Nope
Movem
Nope
Love 
Nope
It's 
Nope
Early
Nope
Early
Nope
i bou
Nope
Might
Nope
Piece
Nope
Early
Nope
Early
Recommend
Early
Recommend
Longe
Nope
Produ
Nope
Early
Nope
No wo
Nope
most 
Recommend
Early
Recommend
rest 
Nope
Early
Nope
This 
Nope
Let m
Nope
It's 
Nope
The g
Recommend
I've 
Recommend
I've 
Recommend
The g
Nope
An ex
Recommend
They 
Nope
It'd 
Nope
The i
Nope
Meh. 
Nope
I lov
Recommend
Cons-
Nope
Who a
Recommend
Love 
Recommend
It is
Nope
Excel
Nope
Love 
Recommend
I lik
Recommend
This 
Recommend
Captu

/home/hithesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
final = pd.concat([df_test['review_id'],df_test['user_suggestion']],axis=1)
final.to_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/nlp_sub2.csv',header=True , index=False)